In [1]:
import pandas as pd
d = pd.read_csv("data/fnd_politifact_claims.csv")
print(d.columns)

# Delete 670
d = d.drop(index=670).reset_index(drop=True)

print("Unique claims:", d.claim.nunique())

Index(['claim', 'claim_factcheck_url', 'claim_author', 'claim_source',
       'claim_date', 'fact_check_date', 'justification',
       'fact_checking_sources', 'issue', 'label'],
      dtype='object')
Unique claims: 2684


In [2]:
# Find all claims with > 1 frequency
duplicate_claims = {}
for idx, claim in enumerate(d.claim):
    matching_rows = d[d.claim == claim]
    if matching_rows.shape[0] > 1:
        row_ids = matching_rows.index.tolist()
        parent_id = row_ids[0]
        child_ids = row_ids[1:]
        duplicate_claims[parent_id] = child_ids

In [3]:
def fetch_links(k, duplicate_claims):
    return [d.iloc[k].claim_factcheck_url, [d.iloc[id].claim_factcheck_url for id in duplicate_claims[k]]]

for k in duplicate_claims.keys():
    label_start = set([d.iloc[k].fact_check_date])
    labels = set()
    for id in duplicate_claims[k]:
        labels.add(d.iloc[id].fact_check_date)

    if label_start != labels:
        print("Discrepancy in labels for claim", k)
        print(label_start, labels)
        print(fetch_links(k, duplicate_claims))

In [4]:
# We need to merge claims. 
# Combine issue into a list

indices_to_drop = set()
for k in duplicate_claims.keys():
    issues = set([d.iloc[k].issue])
    for duplicate_id in duplicate_claims[k]:
        issues.add(d.iloc[duplicate_id].issue)
        indices_to_drop.add(duplicate_id)
    d.iloc[k].issue = str(list(issues))

# Remove the duplicates
d_dropped = d.drop(list(indices_to_drop)).reset_index(drop=True)
len(d_dropped), d.claim.nunique()

d_dropped.to_csv('data/fnd_politifact_claims_final.csv', index=False, encoding='utf-8')

In [1]:
import json
with open("data/claim_queries.json", "r") as f:
    claim_queries = json.load(f)

len(claim_queries)

2684